In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df1 = pd.read_excel('C:/Users/LG/Desktop/4_Cluster Analysis_Data.xlsx', sheet_name="Transactions")

In [3]:
pivotTable = df1.pivot(index="Offer #", columns="Customer Last Name", values="Offer #")
pivotTable = pivotTable.fillna(0).astype(int)
pivotTable[pivotTable > 0] = 1

In [4]:
pivotTable

Customer Last Name,Adams,Allen,Anderson,Bailey,Baker,Barnes,Bell,Bennett,Brooks,Brown,...,Turner,Walker,Ward,Watson,White,Williams,Wilson,Wood,Wright,Young
Offer #,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
7,0,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,1,0,0,0
9,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
trial_list = list()
for n_clusters in range(3, 10):
    temp = list()
    for _ in range(100):
        kmeans = KMeans(n_clusters=n_clusters, max_iter=10000).fit(pivotTable.T)
        temp += [silhouette_score(pivotTable.T, kmeans.labels_, metric='euclidean')]
    trial_list += [temp]

In [6]:
max_list = [0, 0, 0]
for lst in trial_list:
    max_list += [sum(lst) / 50]
print(max_list)
n_clusters = np.argmax(max_list)
print(n_clusters)
kmeans = KMeans(n_clusters=n_clusters, max_iter=10000).fit(pivotTable.T)

[0, 0, 0, 0.2325051321387841, 0.25083519387058234, 0.2450619981198171, 0.24026680076150875, 0.23938229218379178, 0.24354080011066256, 0.24690076167327366]
4


In [7]:
clustedList = [[0 for _ in range(n_clusters)] for _ in range(len(pivotTable.index))]
for i, vi in enumerate(pivotTable.index):
    for j, vj in enumerate(pivotTable.columns):
        clustedList[i][kmeans.labels_[j]] += pivotTable.loc[vi][vj]

In [8]:
df2 = pd.read_excel('C:/Users/LG/Desktop/4_Cluster Analysis_Data.xlsx', sheet_name="OfferInformation")

In [9]:
columns = list()
for i in range(n_clusters):
    columns += ["clus_%s" % i]
clustedDf = pd.DataFrame(data=clustedList, index=pivotTable.index, columns=columns)

In [10]:
df = pd.merge(df2, clustedDf, on="Offer #")
df

,Offer #,Campaign,Varietal,Minimum Qty (kg),Discount (%),Origin,Past Peak,clus_0,clus_1,clus_2,clus_3
0,1,January,Malbec,72,56,France,False,5,3,2,0
1,2,January,Pinot Noir,72,17,France,False,2,6,2,0
2,3,February,Espumante,144,32,Oregon,True,4,0,2,0
3,4,February,Champagne,72,48,France,True,5,3,4,0
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True,0,2,2,0
5,6,March,Prosecco,144,86,Chile,False,8,1,3,0
6,7,March,Prosecco,6,40,Australia,True,3,0,4,12
7,8,March,Espumante,6,45,South Africa,False,3,0,8,9
8,9,April,Chardonnay,144,57,Chile,False,2,2,6,0
9,10,April,Prosecco,72,52,California,False,2,1,4,0


In [11]:
df.sort_values("clus_0", ascending=False)

,Offer #,Campaign,Varietal,Minimum Qty (kg),Discount (%),Origin,Past Peak,clus_0,clus_1,clus_2,clus_3
21,22,August,Champagne,72,63,France,False,20,1,0,0
5,6,March,Prosecco,144,86,Chile,False,8,1,3,0
30,31,December,Champagne,72,89,France,False,8,0,9,0
26,27,October,Champagne,72,88,New Zealand,False,5,1,3,0
10,11,May,Champagne,72,85,France,False,5,2,6,0
0,1,January,Malbec,72,56,France,False,5,3,2,0
3,4,February,Champagne,72,48,France,True,5,3,4,0
29,30,December,Malbec,6,54,France,False,4,0,2,16
2,3,February,Espumante,144,32,Oregon,True,4,0,2,0
6,7,March,Prosecco,6,40,Australia,True,3,0,4,12


In [12]:
print(trial_list)

[[0.09650571216680526, 0.12078917343046144, 0.12122715317053197, 0.1157956147398056, 0.11911443801718427, 0.12133064283950247, 0.11889942863638556, 0.08841691205856658, 0.10694456472824893, 0.12133064283950247, 0.11015391298263667, 0.12078917343046144, 0.11015391298263667, 0.12078917343046144, 0.12133064283950247, 0.12078917343046144, 0.11240643031242324, 0.11806755362739517, 0.11889942863638556, 0.12078917343046144, 0.11852452598572111, 0.12133064283950247, 0.1176869018555452, 0.1176998005869721, 0.12078917343046144, 0.12101628521627421, 0.1176998005869721, 0.1124017526599717, 0.11889942863638556, 0.11806755362739517, 0.12088714398658584, 0.12078917343046144, 0.12133064283950247, 0.11594243674700382, 0.12122715317053197, 0.12078917343046144, 0.11983176694269196, 0.12271860388255451, 0.11015391298263667, 0.12078917343046144, 0.12078917343046144, 0.12078917343046144, 0.11806755362739517, 0.10559256273408764, 0.11272730585043526, 0.08467087154649353, 0.12135935596577617, 0.12078917343046

In [14]:
print(max_list)

[0, 0, 0, 0.2325051321387841, 0.25083519387058234, 0.2450619981198171, 0.24026680076150875, 0.23938229218379178, 0.24354080011066256, 0.24690076167327366]


In [15]:
print(enumerate(pivotTable.index))